# 受限玻尔兹曼机实验
## 1. BernoulliRBM

加载之前预训练好的模型， 参数与下面的设置一样

In [1]:
from rbms import BernoulliRBM
import imageio
from utils import *
brbm = BernoulliRBM(
    pcd=True,
    epoch_start_decay=2,
    epoch_stop_decay=8,
    n_gibbs_steps=2,
    max_epoch=10,
    sample_h_states=False,
    sample_v_states=True,
    batch_size=32,
    h_sz=256,
    l2=0.01)
brbm._load('./pretraineds/brbm_pcd.npz')

True

实验1： 分别从各时间段采样，查看Markov Chain迁移的情况，图片生成的情况。
1. 用随机做h0的初始化。
2. 用测试样本作为v0的初始化。

In [2]:
Batch_sz = 64
X_val = shuffle_batch(mnist_test, Batch_sz)

sto_path_lst = []
val_path_lst = []
for _step in range(20, 2000, 20):
    samples = brbm.inf_by_stochastic(Batch_sz, _step)
    sto_filename = './images/sto_{}.png'.format(_step)
    sto_path_lst.append(sto_filename)
    imsave_(sto_filename,imcombind_(np.reshape(samples, [-1, 28, 28, 1])))
    
    
    samples = brbm.inf_from_valid(X_val, _step)
    val_filename = './images/val_{}.png'.format(_step)
    val_path_lst.append(val_filename)
    imsave_(val_filename,imcombind_(np.reshape(samples, [-1, 28, 28, 1])))

把生成的图片转化成gif动态图，方便查看和展示

In [3]:
to_gif(sto_path_lst, './images/sto_pcd.gif')
to_gif(val_path_lst, './images/val_pcd.gif')

用随机做h0的初始化的实验结果：

In [5]:
from IPython.display import Image
Image(url='./images/sto_pcd.gif')   

用测试样本作为v0的初始化的实验结果：

In [6]:
Image(url='./images/val_pcd.gif') 

### 实验总结
1. 用随机初始化，还是用测试样本初始化，最终模型都能收敛到平稳分布，说明之前的rbm训练ok。
2. 随着Markov_Chain的转移，模型基本能遍历各个model， 多样性很好。
3. 若同样的配置， 把pcd换成cd算法，会失败，模型不能收敛到平稳分布。